<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/Poly_Encoder_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sat Jun 25 05:11:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers datasets folium==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.0 MB/s 
     |████████████████████████████████| 362 kB 71.7 MB/s 
     |████████████████████████████████| 69 kB 9.9 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 6.6 MB 67.6 MB/s 
     |████████████████████████████████| 596 kB 81.1 MB/s 
     |████████████████████████████████| 1.1 MB 75.9 MB/s 
     |████████████████████████████████| 140 kB 65.6 MB/s 
     |████████████████████████████████| 212 kB 77.8 MB/s 
     |████████████████████████████████| 127 kB 8.0 MB/s 
     |████████████████████████████████| 271 kB 98.3 MB/s 
     |████████████████████████████████| 144 kB 71.5 MB/s 
     |████████████████████████████████| 94 kB 5.0 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=9107e71d104de36832affa5577c2625d1b4039e764d0865239a30e2ae8c18480
  S

In [4]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 77 (delta 8), reused 14 (delta 5), pack-reused 57
Unpacking objects: 100% (77/77), done.


In [5]:
!wget https://storage.googleapis.com/bert_models/2020_02_20/all_bert_models.zip
!unzip /content/all_bert_models.zip -d /content/all_bert_models

--2022-06-25 05:11:22--  https://storage.googleapis.com/bert_models/2020_02_20/all_bert_models.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 172.253.117.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2941379946 (2.7G) [application/zip]
Saving to: ‘all_bert_models.zip’

all_bert_models.zip 100%[===================>]   2.74G   133MB/s    in 20s     

2022-06-25 05:11:42 (143 MB/s) - ‘all_bert_models.zip’ saved [2941379946/2941379946]

Archive:  /content/all_bert_models.zip
 extracting: /content/all_bert_models/uncased_L-10_H-128_A-2.zip  
 extracting: /content/all_bert_models/uncased_L-10_H-256_A-4.zip  
 extracting: /content/all_bert_models/uncased_L-10_H-512_A-8.zip  
 extracting: /content/all_bert_models/uncased_L-10_H-768_A-12.zip  
 extracting: /content/all_bert_models/uncased_L-12_H-128_A-2.zip  
 extracting: /content/al

In [6]:
!unzip /content/all_bert_models/uncased_L-4_H-512_A-8.zip -d /content/Poly-Encoder/bert_model

Archive:  /content/all_bert_models/uncased_L-4_H-512_A-8.zip
  inflating: /content/Poly-Encoder/bert_model/bert_model.ckpt.data-00000-of-00001  
  inflating: /content/Poly-Encoder/bert_model/bert_config.json  
  inflating: /content/Poly-Encoder/bert_model/vocab.txt  
  inflating: /content/Poly-Encoder/bert_model/bert_model.ckpt.index  


In [7]:
!gdown --folder 1uC1pSCrh9xlieF60z78QuCg7OxvU9kUs
!mv /content/dstc7/*.json /content/Poly-Encoder/dstc7
!mv /content/dstc7/*.tsv /content/Poly-Encoder/dstc7

Retrieving folder list
Processing file 1YXsj5U-P1nj3ID2ni62vvW4m_4AMYh_v ubuntu_dev_subtask_1.json
Processing file 1I46tBMSYrDCFb0UoequBSl4Uqo0GOVms ubuntu_responses_subtask_1.tsv
Processing file 1tHAe_WGFqQUHmdM28RttlVsYukL69qtg ubuntu_test_subtask_1.json
Processing file 1s2Fz0wQD-YL0tR14PmgdYGuuI7pLarFS ubuntu_train_subtask_1.json
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1YXsj5U-P1nj3ID2ni62vvW4m_4AMYh_v
To: /content/dstc7/ubuntu_dev_subtask_1.json
100% 92.8M/92.8M [00:00<00:00, 214MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I46tBMSYrDCFb0UoequBSl4Uqo0GOVms
To: /content/dstc7/ubuntu_responses_subtask_1.tsv
100% 101k/101k [00:00<00:00, 83.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tHAe_WGFqQUHmdM28RttlVsYukL69qtg
To: /content/dstc7/ubuntu_test_subtask_1.json
100% 18.4M/18.4M [00:00<00:00, 98.8MB/s]
Downloading...
From: https://drive.google.com

In [8]:
%cd /content/Poly-Encoder/bert_model
!bash run.sh
%cd /content

/content/Poly-Encoder/bert_model
Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /content/Poly-Encoder/bert_model/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [512]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [512]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 512]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 512]
Loading TF weight bert/embedd

In [9]:
%cd /content/Poly-Encoder/dstc7
!bash parse.sh
%cd /content

/content/Poly-Encoder/dstc7
tcmalloc: large alloc 1859141632 bytes == 0x246e000 @  0x7fa82b4761e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x5939af 0x594cd3 0x594f8e 0x59526e 0x5bfba0 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fa82b073c87 0x5b621a
tcmalloc: large alloc 1859141632 bytes == 0x71172000 @  0x7fa82b4761e7 0x4a3940 0x52ab72 0x527cf3 0x51d358 0x59358d 0x548c51 0x51566f 0x549576 0x4bcb19 0x59c019 0x59588e 0x595e64 0x4d8924 0x5bfbcb 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fa82b073c87 0x5b621a
/content


In [10]:
%cd /content/Poly-Encoder

/content/Poly-Encoder


In [11]:
!python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture bi
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture poly --poly_m 16
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture cross

Namespace(adam_epsilon=1e-08, architecture='bi', bert_model='bert_model/', eval=False, eval_batch_size=32, fp16=False, fp16_opt_level='O1', gpu=0, gradient_accumulation_steps=1, learning_rate=5e-05, max_contexts_length=128, max_grad_norm=1.0, max_response_length=32, model_type='bert', num_train_epochs=10.0, output_dir='output_dstc7/', poly_m=0, print_freq=100, seed=12345, train_batch_size=32, train_dir='dstc7/', use_pretrain=True, warmup_steps=100, weight_decay=0.01)
Train dir: dstc7/
Output dir: output_dstc7/
Loading parameters from bert_model/pytorch_model.bin
Some weights of the model checkpoint at bert_model/ were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS exp

In [12]:
!python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture bi --eval
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture poly --poly_m 16 --eval
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture cross --eval

Namespace(adam_epsilon=1e-08, architecture='bi', bert_model='bert_model/', eval=True, eval_batch_size=32, fp16=False, fp16_opt_level='O1', gpu=0, gradient_accumulation_steps=1, learning_rate=5e-05, max_contexts_length=128, max_grad_norm=1.0, max_response_length=32, model_type='bert', num_train_epochs=10.0, output_dir='output_dstc7/', poly_m=0, print_freq=100, seed=12345, train_batch_size=32, train_dir='dstc7/', use_pretrain=True, warmup_steps=100, weight_decay=0.01)
Train dir: dstc7/
Output dir: output_dstc7/
Loading parameters from output_dstc7/bi_0_pytorch_model.bin
{'eval_loss': 2.34547645971179, 'R1': 0.481, 'R2': 0.581, 'R5': 0.685, 'R10': 0.77, 'MRR': 0.581299067078589}


In [14]:
!cp -r /content/Poly-Encoder/output_dstc7 '/content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/Poly Encoder'